## Import libraries and define const values

In [7]:
import json
import folium
from geopandas import GeoDataFrame
from pysal.viz.mapclassify import Natural_Breaks
import requests

id_field = 'id'
value_field = 'score'
num_bins = 4
fill_color = 'YlOrRd'
fill_opacity = 0.9
REST_API_ADDRESS= 'http://10.90.46.32:4646/'
Alive_URL = REST_API_ADDRESS + 'alive'
BRS_URL = REST_API_ADDRESS + 'BRS'
Flush_URL = REST_API_ADDRESS + 'flushBuffer'
ChangeProteus_URL = REST_API_ADDRESS + 'changeProteus'

## Identify the areas where start-ups thrive

In [14]:
topk = 11 #
eps = 0.1 # We measure distance in radians, where 1 radian is around 100km, and epsilon is the length of each side of the region
f = "null" # 
dist = True
keywordsColumn = "flags"
keywords = "startup-registroimprese"
keywordsColumn2 = ""
keywords2 = ""
table = "BRSflags"

data = {'topk' : topk, 'eps' : eps, 'f' : f, 'input' : table, "keywordsColumn" : keywordsColumn, "keywords" : keywords,"keywordsColumn2":keywordsColumn2,"keywords2":keywords2,"dist":dist}
response = requests.get(BRS_URL, params=data)
print(response.text)
res = json.loads(response.text)
results_geojson={"type":"FeatureCollection","features":[]}
for region in res:
    results_geojson['features'].append({"type": "Feature", "geometry": { "type": "Point", "coordinates": region['center']},
      "properties": {
        "id": region['rank'],
        "score": region['score']
      }})

[
{
"rank":1,
"center":[9.191005,45.47981],
"score":77.0
}
,{
"rank":2,
"center":[12.50779,41.873835],
"score":35.0
}
,{
"rank":3,
"center":[7.661105,45.064135],
"score":16.0
}
,{
"rank":4,
"center":[14.238015,40.869564999999994],
"score":12.0
}
,{
"rank":5,
"center":[11.382850000000001,44.483135],
"score":9.0
}
,{
"rank":6,
"center":[9.652125,45.671640000000004],
"score":7.0
}
,{
"rank":7,
"center":[11.92423,45.40219000000001],
"score":6.0
}
,{
"rank":8,
"center":[18.183224735000003,40.369488649999994],
"score":6.0
}
,{
"rank":9,
"center":[11.223689069999999,43.809649345],
"score":6.0
}
,{
"rank":10,
"center":[13.353245000000003,38.117855000000006],
"score":6.0
}
,{
"rank":11,
"center":[8.93764,44.41054],
"score":5.0
}
]



### Initialize the map and visualize the output regions

In [15]:
m = folium.Map(
    location=[45.474989560000004,9.205786594999998],
    tiles='Stamen Toner',
    zoom_start=11
)
gdf = GeoDataFrame.from_features(results_geojson['features'])
gdf.crs = {'init': 'epsg:4326'}
gdf['geometry'] = gdf.buffer(data['eps']/2).envelope
threshold_scale = Natural_Breaks(gdf[value_field], k=num_bins).bins.tolist()
threshold_scale.insert(0, gdf[value_field].min())
choropleth = folium.Choropleth(gdf, data=gdf, columns=[id_field, value_field],
                                   key_on='feature.properties.{}'.format(id_field),
                                   fill_color=fill_color, fill_opacity=fill_opacity,
                                   threshold_scale=threshold_scale).add_to(m)
fields = list(gdf.columns.values)
fields.remove('geometry')
tooltip = folium.features.GeoJsonTooltip(fields=fields)
choropleth.geojson.add_child(tooltip)
m